In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker 
import pycountry_convert as pc
import folium
import branca
from datetime import datetime, timedelta,date
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import json, requests

from keras.layers import Input, Dense, Activation, LeakyReLU
from keras import models
from keras.optimizers import RMSprop, Adam

%matplotlib inline

Using TensorFlow backend.


In [82]:
india_data_json = requests.get('https://api.covid19india.org/data.json').json()
#india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()

df_india = pd.io.json.json_normalize(india_data_json['cases_time_series'])
df_india = df_india.set_index("date")
india_data_json

{'cases_time_series': [{'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '30 January ',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '31 January ',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '01 February ',
   'totalconfirmed': '1',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '02 February ',
   'totalconfirmed': '2',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '1',
   'dailydeceased': '0',
   'dailyrecovered': '0',
   'date': '03 February ',
   'totalconfirmed': '3',
   'totaldeceased': '0',
   'totalrecovered': '0'},
  {'dailyconfirmed': '0',
   'dailydeceased': '0',
   'dailyrecovered':

In [83]:
df_india
#total.name = "Total"
#pd.DataFrame(total).style.background_gradient(cmap='summer')

,dailyconfirmed,dailydeceased,dailyrecovered,totalconfirmed,totaldeceased,totalrecovered
date,,,,,,
30 January,1,0,0,1,0,0
31 January,0,0,0,1,0,0
01 February,0,0,0,1,0,0
02 February,1,0,0,2,0,0
03 February,1,0,0,3,0,0
...,...,...,...,...,...,...
02 April,486,16,22,2545,72,191
03 April,560,14,39,3105,86,230
04 April,579,13,56,3684,99,286


In [90]:
total = df_india.sum()
total.name = "Total"
pd.DataFrame(total).transpose().style.background_gradient(cmap='summer',axis=1)

,dailyconfirmed,dailydeceased,dailyrecovered,totalconfirmed,totaldeceased,totalrecovered
Total,1.0011e+104,1.01001e+31,1.001e+65,1.11233e+135,1.12223e+41,1.11222e+87
